In [1]:
%load_ext autoreload
%autoreload 2
    
import os, sys
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import os
from PIL import Image
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from image.models import Image
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet, MIMSAlignment
from PIL import Image
from scipy.ndimage import rotate
import pprint
import sims
import cv2

objc[45539]: Class GNotificationCenterDelegate is implemented in both /Users/chris/miniforge3/envs/emb/lib/libgio-2.0.0.dylib (0x30c376408) and /opt/homebrew/Cellar/glib/2.84.0_1/lib/libgio-2.0.0.dylib (0x3207846c0). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.


In [50]:
import numpy as np
import os
from skimage.io import imread
import matplotlib.pyplot as plt
import tifffile
from PIL import Image

title = "85_mosaic_8"

mims = MIMSImage.objects.filter(file__contains=title).order_by('updated_at').first()
bbox = mims.mims_tiff_images.first().registration_bbox
bbox_tl = [int(min([b[1] for b in bbox])), int(min([b[0] for b in bbox]))]
bbox_br = [int(max([b[1] for b in bbox])), int(max([b[0] for b in bbox]))]

em = tifffile.imread(mims.canvas.images.first().file)
pad_top = max(0, -bbox_tl[0])
pad_left = max(0, -bbox_tl[1])
pad_bottom = max(0, bbox_br[0] - em.shape[0])
pad_right = max(0, bbox_br[1] - em.shape[1])

# Pad the image if needed
if any([pad_top, pad_bottom, pad_left, pad_right]):
    em = np.pad(
        em,
        pad_width=((pad_top, pad_bottom), (pad_left, pad_right)),
        mode='constant',
        constant_values=0
    )
adjusted_tl = [bbox_tl[0] + pad_top, bbox_tl[1] + pad_left]
adjusted_br = [bbox_br[0] + pad_top, bbox_br[1] + pad_left]
print(mims)
em_box = em[adjusted_tl[0]:adjusted_br[0], adjusted_tl[1]:adjusted_br[1]]
print(em_box.shape)


85_ONfast_M1 - 85_mosaic_8.im
(10942, 10942)


In [51]:
import os
import numpy as np
from PIL import Image

# === Output Folder ===
output_folder = f"/Users/chris/Documents/lab/analysis_notebooks/quick_immunoEM_regs/{title}"  # <-- Set this to your actual path
os.makedirs(output_folder, exist_ok=True)

# === Load Images ===
em_array = np.array(em_box)

s_image = mims.mims_tiff_images.filter(name="32S").first().image
au_image = mims.mims_tiff_images.filter(name="197Au").first().image

s_array = np.array(Image.open(s_image))
au_raw = Image.open(au_image)

# === Resize Au to EM shape ===
au_resized = au_raw.resize((em_array.shape[1], em_array.shape[0]), resample=Image.NEAREST)
au_array = np.array(au_resized)

# === Save EM Image ===
em_img = Image.fromarray(em_array.astype(np.uint8))  # ensure 8-bit grayscale
em_img.save(os.path.join(output_folder, "em_image.png"))

# === Save 32S Image ===
s_img = Image.fromarray(s_array.astype(np.uint8))
s_img.save(os.path.join(output_folder, "32s_image.png"))

# === Save Resized 197Au Image ===
au_img = Image.fromarray(au_array.astype(np.uint8))
au_img.save(os.path.join(output_folder, "197au_image_resized.png"))

# === Create Overlay (EM + magenta 197Au mask) ===
au_mask = au_array > 0

# Normalize and convert EM to RGB
em_rgb = np.stack([em_array] * 3, axis=-1).astype(np.uint8)
overlay = em_rgb.copy()
overlay[au_mask] = [255, 0, 255]  # magenta

# Save overlay
overlay_img = Image.fromarray(overlay)
overlay_img.save(os.path.join(output_folder, "em_overlay_197au_magenta.png"))
